# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [4]:
# Your code here

# Scale X_train and X_val using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_train = pd.DataFrame(X_train_scaled, columns=X.columns)
X_val = pd.DataFrame(X_val_scaled, columns=X.columns)

X_train

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces
0,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820
1,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820
2,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820
3,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023
4,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820
...,...,...,...,...,...,...,...,...,...,...
816,-0.532331,-0.099842,-0.509252,-0.510628,-0.897228,-0.808132,-1.353116,-2.214167,-0.274466,0.588023
817,-0.309245,-0.099842,-0.509252,0.514106,0.315353,-0.808132,-0.481708,-0.365525,0.088703,-0.994820
818,0.119419,0.632038,-0.509252,-0.513011,-0.899947,1.684999,0.796096,0.866903,-0.207566,0.588023
819,-0.002718,-0.099842,1.304613,-0.889542,-1.329516,0.783758,-0.290233,-0.365525,-0.852668,-0.994820


In [6]:
# Your code here

# Create a LinearRegression model and fit it on scaled training data
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Calculate a baseline r-squared score on training data
linreg.score(X_train, y_train), linreg.score(X_val, y_val)

(0.7868344817421309, 0.6375622643038104)

## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [13]:
# Your code here

# Set up data structure
interactions = []

# Find combinations of columns and loop over them
column_sets = list(combinations(X.columns, 2))

for (c1, c2) in column_sets:
    # Make copies of X_train and X_val
    train_set = X_train.copy()
    test_set = X_test.copy()
    
    # Add interaction term to data
    train_set['interaction'] = train_set[c1] * train_set[c2]
    test_set['interaction'] = test_set[c1] * test_set[c2]
    
    # Find r-squared score (fit on training data, evaluate on validation data)
    score = LinearRegression().fit(train_set, y_train).score(test_set, y_test)
    
    # Append to data structure
    interactions.append(((c1, c2), score))
    
    
# Sort and subset the data structure to find the top 7
top_interactions = sorted(interactions, key=lambda record: record[1], reverse=True)[:7]
top_interactions

[(('1stFlrSF', 'Fireplaces'), -1426379.252578415),
 (('TotalBsmtSF', 'Fireplaces'), -1532661.4045867252),
 (('GarageArea', 'Fireplaces'), -1674034.5560282203),
 (('OverallQual', 'TotRmsAbvGrd'), -1734068.2188583324),
 (('TotRmsAbvGrd', 'Fireplaces'), -1752987.974196044),
 (('GrLivArea', 'Fireplaces'), -1837952.886961307),
 (('OverallQual', 'Fireplaces'), -1883381.2955321842)]

### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [15]:
# Your code here

# Loop over top 7 interactions
for record in top_interactions:

    # Extract column names from data structure
    c1, c2 = record[0]

    # Construct new column name
    col_name = c1 + '_' + c2
    
    # Add new column to X_train and X_val
    X_train[col_name] = X_train[c1] * X_train[c2]
    X_val[col_name] = X_val[c1] * X_val[c2]

X_train

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,1stFlrSF_Fireplaces,TotalBsmtSF_Fireplaces,GarageArea_Fireplaces,OverallQual_TotRmsAbvGrd,TotRmsAbvGrd_Fireplaces,GrLivArea_Fireplaces,OverallQual_Fireplaces
0,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820,0.800620,0.636004,-0.325932,-0.025029,-0.249390,-0.496529,0.099325
1,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820,-0.638285,-0.833866,-0.078736,-0.231026,0.363632,0.245968,-0.628764
2,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820,0.327296,0.012495,1.100202,0.816535,0.976654,0.939872,0.827414
3,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023,-0.358127,-0.199366,-0.667173,0.816535,-0.577286,-0.673881,-0.489072
4,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820,1.309105,2.518386,2.269632,-0.391978,-0.249390,0.479213,1.555503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,-0.532331,-0.099842,-0.509252,-0.510628,-0.897228,-0.808132,-1.353116,-2.214167,-0.274466,0.588023,-0.527591,-0.300261,-0.161392,0.221068,-1.301982,-0.795664,-0.058710
817,-0.309245,-0.099842,-0.509252,0.514106,0.315353,-0.808132,-0.481708,-0.365525,0.088703,-0.994820,-0.313720,-0.511443,-0.088243,0.036495,0.363632,0.479213,0.099325
818,0.119419,0.632038,-0.509252,-0.513011,-0.899947,1.684999,0.796096,0.866903,-0.207566,0.588023,-0.529190,-0.301663,-0.122054,0.547915,0.509759,0.468123,0.371653
819,-0.002718,-0.099842,1.304613,-0.889542,-1.329516,0.783758,-0.290233,-0.365525,-0.852668,-0.994820,1.322629,0.884934,0.848252,0.036495,0.363632,0.288730,0.099325


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [30]:
# Your code here

# Set up data structure
polynomials = []

# Loop over all columns
for column in X_train.columns:

    # Loop over degrees 2, 3, 4
    for degree in [2, 3, 4]:
        
        # Make a copy of X_train and X_val
        features_train = X_train.copy()
        features_val = X_val.copy()
    
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(degree)
        
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        transformed_train = pd.DataFrame(poly.fit_transform(features_train[[column]]))
        transformed_val = pd.DataFrame(poly.fit_transform(features_val[[column]]))

        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
        features_train = pd.concat([features_train.drop(column, axis=1), transformed_train], axis=1)
        features_val = pd.concat([features_val.drop(column, axis=1), transformed_val], axis=1)
    
        # Find r-squared score on validation
        score = LinearRegression().fit(features_train, y_train).score(features_val, y_val)
    
        # Append to data structure
        polynomials.append(((column, degree, score)))

# Sort and subset the data structure to find the top 7
top_polynomials = sorted(polynomials, key=lambda p: p[-1], reverse=True)[:7]
top_polynomials


[('GrLivArea', 4, 0.840566487625741),
 ('GarageArea_Fireplaces', 3, 0.8378978005115225),
 ('TotRmsAbvGrd_Fireplaces', 4, 0.836631969861492),
 ('1stFlrSF', 2, 0.6276363368875173),
 ('GrLivArea', 3, 0.6200292893882771),
 ('TotalBsmtSF', 2, 0.5754934763044448),
 ('TotRmsAbvGrd_Fireplaces', 3, 0.5645798507606035)]

### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [31]:
# Your code here
top_polynomials = pd.DataFrame(top_polynomials, columns=["Column", "Degree", "R^2"])

# Drop duplicate columns based on Column name
top_polynomials.drop_duplicates(subset="Column", inplace=True)

# Loop over remaining results
for (col, degree, _) in top_polynomials.values:
    #Create polynomimal terms
    poly = PolynomialFeatures(degree, include_bias=False)
    train_poly = pd.DataFrame(poly.fit_transform(X_train[[col]]),
                              columns=poly.get_feature_names([col]))
    val_poly = pd.DataFrame(poly.transform(X_val[[col]]),
                              columns=poly.get_feature_names([col]))
    
    #Concat back to original
    X_train = pd.concat([X_train.drop(col, axis=1), train_poly], axis=1)
    X_val = pd.concat([X_val.drop(col, axis=1), val_poly], axis=1)
    

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [32]:
# Your code here
X_train.head()

,LotArea,OverallQual,OverallCond,2ndFlrSF,TotRmsAbvGrd,GarageArea,Fireplaces,1stFlrSF_Fireplaces,TotalBsmtSF_Fireplaces,OverallQual_TotRmsAbvGrd,...,GarageArea_Fireplaces^2,GarageArea_Fireplaces^3,TotRmsAbvGrd_Fireplaces,TotRmsAbvGrd_Fireplaces^2,TotRmsAbvGrd_Fireplaces^3,TotRmsAbvGrd_Fireplaces^4,1stFlrSF,1stFlrSF^2,TotalBsmtSF,TotalBsmtSF^2
0,-0.114710,-0.099842,-0.509252,1.261552,0.250689,0.327629,-0.994820,0.800620,0.636004,-0.025029,...,0.106232,-0.034624,-0.249390,0.062195,-0.015511,0.003868,-0.804789,0.647685,-0.639316,0.408725
1,-0.176719,0.632038,-0.509252,-0.808132,-0.365525,0.079146,-0.994820,-0.638285,-0.833866,-0.231026,...,0.006199,-0.000488,0.363632,0.132228,0.048082,0.017484,0.641608,0.411661,0.838208,0.702592
2,-0.246336,-0.831723,1.304613,-0.808132,-0.981739,-1.105931,-0.994820,0.327296,0.012495,0.816535,...,1.210445,1.331733,0.976654,0.953852,0.931583,0.909834,-0.329000,0.108241,-0.012560,0.000158
3,-0.378617,-0.831723,1.304613,-0.808132,-0.981739,-1.134602,0.588023,-0.358127,-0.199366,0.816535,...,0.445119,-0.296971,-0.577286,0.333259,-0.192385,0.111061,-0.609036,0.370925,-0.339045,0.114951
4,-0.010898,-1.563603,1.304613,0.550523,0.250689,-2.281450,-0.994820,1.309105,2.518386,-0.391978,...,5.151230,11.691396,-0.249390,0.062195,-0.015511,0.003868,-1.315922,1.731650,-2.531499,6.408489


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [34]:
# Your code here
linreg = LinearRegression()
linreg.fit(X_train, y_train)

print(f'Training Set R²: {linreg.score(X_train, y_train)}')
print(f'Test Set R²: {linreg.score(X_val, y_val)}')

Training Set R²: 0.8491535501803753
Test Set R²: -1.6223683879867723


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [38]:
# Your code here
for n in range(5, 21, 5):
    rfe = RFE(LinearRegression(), n_features_to_select=n)
    X_rfe_train = rfe.fit_transform(X_train, y_train)
    X_rfe_val = rfe.transform(X_val)

    lr = LinearRegression()
    lr.fit(X_rfe_train, y_train)

    print(f"Using {n} features")
    print("Train R^2:", lr.score(X_rfe_train, y_train))
    print("Test R^2: ", lr.score(X_rfe_val, y_val))
    print()
    

Using 5 features
Train R^2: 0.761338610096933
Test R^2:  0.6145963635536846

Using 10 features
Train R^2: 0.8265527369041241
Test R^2:  0.6763957022287043

Using 15 features
Train R^2: 0.8366074146143467
Test R^2:  0.4228738183823503

Using 20 features
Train R^2: 0.8392835025666003
Test R^2:  0.4723686248222603



Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [49]:
# Your code here

train_mse = []
test_mse = []
alphas = np.linspace(0, 90000, num=5)

for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    
    print(f"With an alpha of {alpha}")
    print("Train R^2:", lasso.score(X_train, y_train))
    print("Test R^2: ", lasso.score(X_val, y_val))
    print()


With an alpha of 0.0
Train R^2: 0.849153548639979
Test R^2:  -1.623268145910822

With an alpha of 22500.0
Train R^2: 0.6435613302649067
Test R^2:  -0.5634119122095225

With an alpha of 45000.0
Train R^2: 0.3421428613320793
Test R^2:  0.30591791786680145

With an alpha of 67500.0
Train R^2: 0.1720176088022709
Test R^2:  -0.7967892024704473

With an alpha of 90000.0
Train R^2: 0.1327045978806951
Test R^2:  -2.3842953940921348



Compare the results. Which features would you choose to use?

In [ ]:
# Your written answer here

# For an RFE model, I would use 10 features. For a Lasso model, the best alpha was about 45k. In this scenario I would use an RFE model

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [50]:
# Your code here
X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled, columns=X.columns)

for int in top_interactions:
    c1, c2 = int[0]
    new_col_name = c1 + "_" + c2
    X_test[new_col_name] = X_test[c1] * X_test[c2]

for (col, degree, _) in top_polynomials.values:
    poly = PolynomialFeatures(degree, include_bias=False)
    poly_test = pd.DataFrame(poly.fit_transform(X_test[[col]]),
                                        columns=poly.get_feature_names([col]))
    X_test = pd.concat([X_test.drop(col, axis=1), poly_test], axis=1)

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [51]:
# Your code here
final_model = Lasso(alpha=10000)
final_model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

print("R-Squared:", final_model.score(X_test, y_test))
print("MSE:", mean_squared_error(y_test, final_model.predict(X_test)))

R-Squared: 0.7970742584094077
MSE: 1421557534.5172462


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.